# Assignment for Capstone Project

Import all the libraries

In [14]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import geocoder
import io
import requests
import folium

Using BeautifulSoup for webscraping

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
#print(soup.prettify())

In [3]:
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable')

Creating the dataframe using the table created from 'wikitable'

In [4]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)== 3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C

Cleaning the dataframe

In [5]:
df = df.drop(df[df.Borough == 'Not assigned\n'].index)
df.reset_index(drop=True)
df['Neighborhood']= df.groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x : ', '.join(x))
df = df.replace('\n','', regex=True)
df = df.replace(' / ',', ', regex=True)
df = df.reset_index(drop=True)

In [6]:
df.shape

(103, 3)

Finding latitude and longitude for postal code

In [7]:
# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
 # g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

Loading csv file

In [8]:
url2 = 'http://cocl.us/Geospatial_data'
s = requests.get(url2).content
coordonates = pd.read_csv(io.StringIO(s.decode('utf-8')))
coordonates = coordonates.rename(columns={"Postal Code": "PostalCode"})

Merge dataframes

In [9]:
df = pd.merge(df, coordonates, on ='PostalCode')

Keep Boroughs that contain 'Toronto'

In [12]:
df_toronto = df[df['Borough'].str.contains("Toronto")]

# Visualize the neighborhoods of Toronto

In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto